# **Processamento de Linguagem Natural [2024-Q2]**
Prof. Alexandre Donizeti Alves

### **PROJETO PRÁTICO** [LangChain + Grandes Modelos de Linguagem + API]


O **PROJETO PRÁTICO** deve ser feitO utilizando o **Google Colab** com uma conta sua vinculada ao Gmail. O link do seu notebook, armazenado no Google Drive, o link de um repositório no GitHub e o link de um vídeo do projeto em execução detalhando os principais resultados da atividade, devem ser enviados usando o seguinte formulário:

> https://forms.gle/D4gLqP1iGgyn2hbH8


**IMPORTANTE**: A submissão deve ser feita até o dia **08/09 (domingo)** APENAS POR UM INTEGRANTE DA EQUIPE, até às 23h59. Por favor, lembre-se de dar permissão de ACESSO IRRESTRITO para o professor da disciplina de PLN.

### **EQUIPE**

---

**POR FAVOR, PREENCHER OS INTEGRANDES DA SUA EQUIPE:**


**Integrante 01:**

`André Ribeiro Thasmo - 11202130043`

**Integrante 02:**

`Bheatriz Almeida Santos de Jesus - 11202131693`

**Integrante 03:**

`Caio Alexandre Sampaio Diniz - 11202130822`

### **GRANDE MODELO DE LINGUAGEM (*Large Language Model - LLM*)**

---

Cada equipe deve selecionar um Grande Modelo de Linguagem (*Large Language Model - LMM*). Cada modelo pode ser escolhido por até 5 equipes.



Por favor, informe os dados do LLM selecionada:

>


**LLM**: OpenAI(GPT-4o-mini)

>

**Link para a documentação oficial**: [Documentação OPenIA](https://platform.openai.com/docs/models/gpt-4o%23.ppt)




### **API**
---

Por favor, informe os dados da API selecionada:

**API**: TheMovieDB

**Site oficial**: https://www.themoviedb.org/?language=pt-BR

**Link para a documentação oficial**: [Documentação TheMoviesDB](https://developer.themoviedb.org/reference/intro/getting-started)





**IMPORTANTE**: cada **API** pode ser usada por até 4 equipes.

### **DESCRIÇÃO**
---

Implementar um `notebook` no `Google Colab` que faça uso do framework **`LangChain`** (obrigatório) e de um **LLM** aplicando, no mínimo, DUAS técnicas de PLN. As técnicas podem ser aplicada em qualquer córpus obtido a partir de uma **API** ou a partir de uma página Web.

O **LLM** e a **API** selecionados devem ser informados na seguinte planilha:

> https://docs.google.com/spreadsheets/d/1iIUZcwnywO7RuF6VEJ8Rx9NDT1cwteyvsnkhYr0NWtU/edit?usp=sharing

>
As seguintes técnicas de PLN podem ser usadas:

*   Correção Gramatical
*   Classificação de Textos
*   Análise de Sentimentos
*   Detecção de Emoções
*   Extração de Palavras-chave
*   Tradução de Textos
*   Sumarização de Textos
*   Similaridade de Textos
*   Reconhecimento de Entidades Nomeadas
*   Sistemas de Perguntas e Respostas
>

**IMPORTANTE:** É obrigatório usar o e-mail da UFABC.


### **CRITÉRIOS DE AVALIAÇÃO**
---


Serão considerados como critérios de avaliação os seguintes pontos:

* Uso do framework **`LangChain`**.

* Escolha e uso de um **LLM**.

* Escolha e uso de uma **API**.

* Vídeo (5 a 10 minutos).

* Criatividade no uso do framework **`LangChain`** em conjunto com o **LLM** e a **API**.




**IMPORTANTE**: todo o código do notebook deve ser executado. Código sem execução não será considerado.

### **IMPLEMENTAÇÃO**
---


#### **Instalando Dependências**

In [1]:
# Instalando a biblioteca da API da OpenAI
!pip install openai

# Instalando o pacote LangChain
!pip install -qU langchain

# Integração com o pacote da OpenAI
!pip install -qU langchain-openai

# Instalando o pacote do LangChain Community
!pip install langchain_community

#### Definindo Chave da API da OpenAI

In [2]:
# Definindo a chave da API
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass()

··········


#### Extraindo Dados dos 1000 Filmes mais Populares com a TMDB API

In [3]:
import requests

# Insira sua chave de API do TMDB
API_KEY = 'fd860ba3bed232fb601ce8f03c36f690'
API_URL = 'https://api.themoviedb.org/3/movie/popular'

def buscar_filmes_populares(pagina):
    params = {
        'api_key': API_KEY,
        'language': 'pt-BR',
        'page': pagina
    }
    response = requests.get(API_URL, params=params)
    return response.json()

def buscar_1000_filmes_mais_populares():
    todos_os_filmes = []
    pagina_atual = 1
    paginas_maximas = 25

    while pagina_atual <= paginas_maximas:
        resposta = buscar_filmes_populares(pagina_atual)
        if 'results' in resposta:
            todos_os_filmes.extend(resposta['results'])
            total_paginas = resposta.get('total_pages', paginas_maximas)
            if pagina_atual >= total_paginas:
                break
        else:
            print(f"Chave 'results' não encontrada na resposta: {resposta}")
            break
        pagina_atual += 1

    return todos_os_filmes

filmes = buscar_1000_filmes_mais_populares()

#### Criando um KB

In [4]:
import pandas as pd

descricao_filme = []

for filme in filmes:
    # Removendo filmes sem sinopse
    if filme['overview'] != '':
        descricao_filme.append({'titulo': filme['title'], 'sinopse': filme['overview']})

df_filmes = pd.DataFrame(descricao_filme, columns=['titulo', 'sinopse'])

df_filmes.head()


,titulo,sinopse
0,Deadpool & Wolverine,Um apático Wade Wilson trabalha duro na vida c...
1,Borderlands: O Destino do Universo Está em Jogo,Uma infame caçadora de recompensas retorna ao ...
2,Divertida Mente 2,"""Divertida Mente 2"", da Disney e da Pixar, ret..."
3,Os Fantasmas Ainda se Divertem: Beetlejuice Be...,"Depois de uma tragédia familiar inesperada, tr..."
4,Crepúsculo dos Guerreiros: Walled In,"Na década de 1980, o jovem problemático Chan L..."


#### Classificando Gênero dos Filmes

In [5]:
from langchain_core.pydantic_v1 import BaseModel, Field

class Classificacao_Genero(BaseModel):
    genero: str = Field(description="O gênero do filme")

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template(
    """
Extraia as informações desejadas do seguinte texto.

Extraia apenas as propriedades mencionadas em 'Classificacao_Genero'.

Texto:
{input}
"""
)

In [7]:
modelo = ChatOpenAI(temperature=0, model="gpt-4o-mini").with_structured_output(Classificacao_Genero)

chain = prompt | modelo

##### Criando uma nova coluna no KB com o gênero determinado pelo modelo

In [8]:
for row in df_filmes.itertuples():
    resposta = chain.invoke({"input": f"Titulo: {row.titulo} Sinopse: {row.sinopse}"})
    df_filmes.at[row.Index, 'genero'] = resposta.genero.capitalize()

In [9]:
df_filmes.head()

,titulo,sinopse,genero
0,Deadpool & Wolverine,Um apático Wade Wilson trabalha duro na vida c...,Ação
1,Borderlands: O Destino do Universo Está em Jogo,Uma infame caçadora de recompensas retorna ao ...,Ação
2,Divertida Mente 2,"""Divertida Mente 2"", da Disney e da Pixar, ret...",Animação
3,Os Fantasmas Ainda se Divertem: Beetlejuice Be...,"Depois de uma tragédia familiar inesperada, tr...",Comédia
4,Crepúsculo dos Guerreiros: Walled In,"Na década de 1980, o jovem problemático Chan L...",Ação


#### Sistema de Recomendação de Filmes

In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI



# Inicializando o modelo
modelo = ChatOpenAI(temperature=0, model="gpt-4o-mini")

# Definindo o prompt do sistema
system_prompt = (
    "Você é um assistente para fazer recomendações de filmes."
    " Dado o DataFrame df_filmes como contexto, recomende filmes com base na sinopse."
    " Recomende 3 filmes com base na descrição do usuário."
    " Não recomende nenhum filme que não esteja no DataFrame df_filmes."
    " Dê a sua resposta em Markdown e utilize emojis"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input_usuario}"),
    ]
)

chain = prompt | modelo

input_usuario = input("Faça uma breve descrição de algo que você queira assistir: ")

resposta = chain.invoke({"input_usuario": input_usuario, "context": df_filmes.to_markdown()})

print()

# Formatando o output para MarkDown
from IPython.display import Markdown, display
display(Markdown(resposta.content))

Faça uma breve descrição de algo que você queira assistir: algo com muita ação e tiros



Aqui estão três filmes cheios de ação e tiros que você pode gostar! 🎬💥

1. **Deadpool & Wolverine**  
   Um apático Wade Wilson trabalha duro na vida civil. Seus dias como mercenário moralmente flexível, Deadpool, ficaram para trás. Quando seu planeta enfrenta uma ameaça, Wade deve relutantemente vestir-se novamente com um ainda mais relutante... relutante? Mais relutante? Ele deve convencer um Wolverine relutante em... Porra. As sinopses são tão estúpidas. 

2. **Bad Boys: Até o Fim**  
   Depois que seu falecido ex-capitão é incriminado, Lowrey e Burnett tentam limpar seu nome, apenas para acabarem se tornando os mais procurados de Miami. 

3. **KILL: O Massacre no Trem**  
   Uma viagem de trem por Nova Deli, capital da Índia. O cenário logo se transforma em um campo de batalha quando uma dupla de soldados enfrenta um exército de bandidos invasores. 

Aproveite a adrenalina! 🍿🔥